# Parquet Files (Optional)

As we mentioned in <a href="Data File Types.ipynb">Data File Types</a>, `.parquet` files are a useful data file type. They can help improve query speed, decrease memory requirements, and speed up column based calculations.

## What we will accomplish

In this notebook we will:
- Dive a little deeper into the `.parquet` file type,
- Learn a bit more about how `.parquet` files improve query performance,
- Talk about the `pyarrow` package, and
- Demonstrate concepts with an actual `.parquet` file.

In [1]:
import pandas as pd

In the following notebook we will use a fictional data set, `study_results`, that contains the results of a study examining the effectiveness of three treatments on reducing recovery time in adults. We can load the `.csv` version of these data with `read_csv`. We will eventually look at a parquet equivalent of this file later in the notebook.

In [2]:
df_csv = pd.read_csv("../data/study_results.csv")

df_csv.head()

,sex,treatment,age,recovery
0,F,A,19,12
1,F,A,21,2
2,F,A,22,13
3,F,A,24,12
4,F,A,28,3


In [3]:
len(df_csv)

120

## Row format compared to columnar format

Recall that file types like `.csv`s and `.tsv`s store data in a row-based format like so:

<img src="row_based.png" width="40%"></img>

and `.parquet` files store their data in a columnar format like so:

<img src="columnar.png" width="60%"></img>

Above, this columnar format was hidden from us, in a sense, by `pandas` because `read_parquet` automatically converts the `.parquet` file into a `pandas` `DataFrame` which has a row-based approach to storing data. However, the columnar approach gives `.parquet` files its most desirable quality for data analysts and scientists, increased column querying speed.

### Increasing column query speed

Many real world applications have data sets with many columns (think hundreds or more), but you will commonly only want to use relatively few columns for any given analysis or application. Row-based formats are slow to subselect a set of columns because you need to traverse each row (of which there may be millions) and select the appropriate values. By contrast, with columnar formats you only need to traverse the set of columns and choose the ones you want, this is much faster because there are typically much fewer columns than rows.

## Actually directories

Up to this point we have thought of parquet files as single files. While there are distinct `.parquet` files, it is more common in practice to deal with the parquet format as a directory of `.parquet` files.

### Partitioning

For a given data set a single `.parquet` file tends to only store a subset of the data. This subset is formed through partitioning. For example, the study data are split according to sex and then according to treatment group as described in the schematic below.


<img src="partition.png" width="20%"></img>

We can implement such a partitioning using `to_parquet` along with the `partition_cols` argument which takes in a `list` of column names along which the data are partitioned. Let's do that now with our `study_data.csv` data.

In [4]:
## Note that when we do this we are naming a directory
## NOT a file, so there is no .parquet file at the end
## of the file name
df_csv.to_parquet("../data/study_data_parquet/",
                  partition_cols=['sex', 'treatment'],
                  index=False)

Now if you go to the `data` folder you will see the folder `study_data_parquet` in there. Inside that folder are `sex=M` and `sex=F` folders, each of which contain `treatment=A`, `treatment=B`, `treatment=C` folders as demonstrated in the sequential images below:

In the `data` folder:
<img src="study_parq_folder_1.png" width="90%"></img>

In the `data/study_data_parquet` folder:
<img src="study_parq_folder_2.png" width="90%"></img>

In the `data/study_data_parquet/sex=F` folder:
<img src="study_parq_folder_3.png" width="90%"></img>

In the `data/study_data_parquet/sex=F/treatment=A` folder:
<img src="study_parq_folder_4.png" width="90%"></img>

Even though the parquet "file" is a directory you can still read it in using `read_parquet`.

In [5]:
df_parq = pd.read_parquet("../data/study_data_parquet/")

In [6]:
df_parq

,age,recovery,sex,treatment
0,19,12,F,A
1,21,2,F,A
2,22,13,F,A
3,24,12,F,A
4,28,3,F,A
...,...,...,...,...
115,59,14,M,C
116,61,11,M,C
117,61,13,M,C
118,62,8,M,C


However, as we can see, this loads the entire data set as a single table, which may be undesirable if the data set is quite large. Luckily, there are alternative ways to loading a parquet file in python.

## `pyarrow`

One way to take greater advantage of the benefits of parquet is by using the `pyarrow` package, <a href="https://arrow.apache.org/docs/python/parquet.html">https://arrow.apache.org/docs/python/parquet.html</a>, directly. First we need to import `parquet` from `pyarrow`

In [7]:
## Import parquet as pq
import pyarrow.parquet as pq

Next we can load the parquet directory with `pq.ParquetDataset`.

In [9]:
## You can read in a directory with
## pq.ParquetDataset(directory name)
study_directory = pq.ParquetDataset("../data/study_data_parquet/")

We can see how the parquet directory was partitioned using `.partitioning.dictionaries`.

In [10]:
study_directory.partitioning.dictionaries

 [
   "F",
   "M"
 ],
 [
   "A",
   "B",
   "C"
 ]]

You can get the data using the `.read()` function. You can then turn it into a `pandas` `DataFrame` using `.to_pandas()`.

In [11]:
## Creating a pyarrow Table object
study_directory.read()

pyarrow.Table
age: int64
recovery: int64
sex: dictionary<values=string, indices=int32, ordered=0>
treatment: dictionary<values=string, indices=int32, ordered=0>
----
age: [[19,21,22,24,28,...,56,58,58,59,63],[22,24,24,24,25,...,50,57,64,64,64],...,[19,19,19,22,26,...,48,49,52,55,56],[19,21,23,24,25,...,59,61,61,62,64]]
recovery: [[12,2,13,12,3,...,12,13,14,10,3],[9,4,13,13,10,...,12,13,4,10,13],...,[3,9,10,8,12,...,6,6,9,3,14],[2,4,10,3,7,...,14,11,13,8,7]]
sex: [  -- dictionary:
["F","M"]  -- indices:
[0,0,0,0,0,...,0,0,0,0,0],  -- dictionary:
["F","M"]  -- indices:
[0,0,0,0,0,...,0,0,0,0,0],...,  -- dictionary:
["F","M"]  -- indices:
[1,1,1,1,1,...,1,1,1,1,1],  -- dictionary:
["F","M"]  -- indices:
[1,1,1,1,1,...,1,1,1,1,1]]
treatment: [  -- dictionary:
["A","B","C"]  -- indices:
[0,0,0,0,0,...,0,0,0,0,0],  -- dictionary:
["A","B","C"]  -- indices:
[1,1,1,1,1,...,1,1,1,1,1],...,  -- dictionary:
["A","B","C"]  -- indices:
[1,1,1,1,1,...,1,1,1,1,1],  -- dictionary:
["A","B","C"]  -- in

In [12]:
## Turning a pyarrow table into a dataframe
study_directory.read().to_pandas()

,age,recovery,sex,treatment
0,19,12,F,A
1,21,2,F,A
2,22,13,F,A
3,24,12,F,A
4,28,3,F,A
...,...,...,...,...
115,59,14,M,C
116,61,11,M,C
117,61,13,M,C
118,62,8,M,C


If we want to filter the data before it is loaded, we can add a `filters` argument to `pq.ParquetDataset`.

In [15]:
## filters takes in a list of tuples
## each tuple has a logic condition
## (the column string, the logical comparison, the value for comparison)
## subsetting is performed in order of the list
study_directory_F = pq.ParquetDataset("../data/study_data_parquet/",
                                         filters=[('sex', '==', 'F'),
                                                  ('age', '<', 50)])

In [16]:
## examine the filter in action
study_directory_F.read().to_pandas().sex.value_counts()

F    40
M     0
Name: sex, dtype: int64

In [18]:
study_directory_F.read().to_pandas().age.describe()

count    40.000000
mean     32.950000
std       8.218023
min      19.000000
25%      25.000000
50%      32.500000
75%      39.250000
max      48.000000
Name: age, dtype: float64

There are also ways to filter after you have loaded the directory, but we will touch on that in the corresponding practice problems notebook. If you are interested in learning more about how you can manage parquet files with `pyarrow`, check out their documentation:
- Goes directly to handling parquet files: <a href="https://arrow.apache.org/docs/python/parquet.html">https://arrow.apache.org/docs/python/parquet.html</a>,
- Starts at the beginning of the documentation: <a href="https://arrow.apache.org/docs/python/getstarted.html">https://arrow.apache.org/docs/python/getstarted.html</a>.

That will be it for this notebook!

--------------------------

This notebook was written for the Erd&#337;s Institute C&#337;de Data Science Boot Camp by Matthew Osborne, Ph. D., 2023.

Any potential redistributors must seek and receive permission from Matthew Tyler Osborne, Ph.D. prior to redistribution. Redistribution of the material contained in this repository is conditional on acknowledgement of Matthew Tyler Osborne, Ph.D.'s original authorship and sponsorship of the Erdős Institute as subject to the license (see License.md)